In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
# Import Python packages
import pandas as pd
import os
import json
import pickle
import numpy as np
import time
import sys
# import cachetools
# Import Snowflake modules
from snowflake.snowpark import Session, DataFrame
from snowflake.snowpark import functions as F
from snowflake.snowpark import types as T
from snowflake.snowpark import Row, Column
import pandas as pd
from snowflake.snowpark.version import VERSION
from snowflake.snowpark.types import StringType,IntegerType

In [ ]:
# Add a query tag to the session.
session.query_tag = {"origin":"sf_sit-is", 
                     "name":"all_center_analytics", 
                     "version":{"major":1, "minor":0},
                     "attributes":{"is_quickstart":1, "source":"notebook"}}
snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()

In [ ]:
alter stage AUDIO_FILES refresh;

In [ ]:
INSERT INTO ALL_CLAIMS_RAW
        (
            DATETIME,
            AUDIOFILE,
            AUDIOFILE_RELATIVE_PATH,
            CONVERSATION,
            DURATION,
            PRESIGNED_URL_PATH
        )
        SELECT 
            CAST(CASE WHEN split(RELATIVE_PATH,'/')[1]::string IS NULL THEN GETDATE() 
                      ELSE split(RELATIVE_PATH,'/')[0]::string END AS DATE) as Datetime,
            CASE WHEN split(RELATIVE_PATH,'/')[1]::string is null then split(RELATIVE_PATH,'/')[0]::string 
                  ELSE split(RELATIVE_PATH,'/')[1]::string END as AudioFile,
            RELATIVE_PATH AS   AUDIOFILE_RELATIVE_PATH,    
            AI_TRANSCRIBE(TO_FILE('@AUDIO_FILES', RELATIVE_PATH))['text']::varchar AS EXTRACTED_TEXT,
            AI_TRANSCRIBE(TO_FILE('@AUDIO_FILES', RELATIVE_PATH))['audio_duration']::varchar AS DURATION_IN_SECONDS,
            GET_PRESIGNED_URL('@AUDIO_FILES', RELATIVE_PATH) AS PRESIGNED_URL
        FROM DIRECTORY('@AUDIO_FILES') ;

In [ ]:
generated_transcripts_df = session.table("ALL_CLAIMS_RAW")

In [ ]:
print(generated_transcripts_df.limit(3).show())

In [ ]:
# @F.udf(name="get_qa_prompt_oneshort", session=session, packages=['scikit-learn==1.1.1', 'pandas'], is_permanent=True, stage_location="@udf", replace=True, input_types=[T.PandasSeriesType(T.StringType())], return_type=T.PandasSeriesType(T.StringType()))
# def get_qa_prompt_oneshort(query_series:pd.Series):
#     response_series =  query_series.map(lambda query: f"""
                                        
# <s>[INST]
# <<SYS>>
# You are a expert in analyzing customer transcripts and tasked with answering questions about an article.  
# Please ensure that your responses are socially unbiased and positive in nature.
# If you don't know the answer, please don't share false information.

# Output answer in the following format and no preamble. :
#                                         * <Representative> : text of the Representative
#                                         * <Customer>  : text of the customer
# Extract the name of the Representative name from the transcript 
# and replace that name for <Representative> and extract Customer name from the trasnscript replace that with <Customer> for creating the conversation. 

# Below is one example output for the conversation:   
                                                                    
# Representative: Jason
# Customer: Michelle Turner

# Jason: Hi, Michelle. My name is Jason from AutoAssure Insurance. How may I assist you today?
# Michelle: Hi, Jason. I'm calling regarding a dispute I have with a recent claim.
# Jason: Of course, Michelle. Can you please provide me with your claim number so I can access your file?
# Michelle: Certainly, it's CLM876543.
# Jason: Thank you, Michelle. Let me retrieve your claim information. While I'm doing that, could you explain the nature of the dispute?
# Michelle: Of course, Jason. I believe there's been a misunderstanding in the assessment of the damages to my vehicle.
# Jason: I appreciate you providing details, Michelle. I'll review your claim thoroughly and work towards a resolution.
# Michelle: Thank you, Jason. I hope we can reach a fair resolution.
# Jason: We'll do our best, Michelle. I'll keep you updated on the progress. Have a good day!
                                                                              
# <</SYS>>                                                                                                             
# We need to analyze the text of the transcript found in <transcript> </transcript> tag as conversation between representative and the customer. Convert the transcript in the format mentioned in the example. 
# Extract the name of the Representative name from the transcript and replace that name for <Representative> and extract Customer name from the trasnscript replace that with <Customer> while creating the conversation.                                       
# Do not include "Sure, here is the conversation between in the output. Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly". 
# <transcript>
#      {query}
# </transcript> 


# [/INST] Answer: Sure, I would be happy to help! """                                        
#     )

#     return response_series

In [ ]:
@F.udf(name="get_qa_prompt_oneshort", session=session, packages=['scikit-learn==1.1.1', 'pandas'], is_permanent=True, stage_location="@udf", replace=True, input_types=[T.PandasSeriesType(T.StringType())], return_type=T.PandasSeriesType(T.StringType()))
def get_qa_prompt_oneshort(query_series:pd.Series):
    response_series = query_series.map(lambda query: f"""
            [INST] Convert this transcript to a structured conversation format:
            Representative: [Name]
            Customer: [Name]
            
            [Conversation lines...]
            
            <transcript>{query}</transcript>
            [/INST]""")
    return response_series

In [ ]:
oneshort_prompted_snowdf = generated_transcripts_df.with_column("Oneshort_PROMPT", F.call_udf("get_qa_prompt_oneshort", F.col("CONVERSATION")))

In [ ]:
oneshort_prompted_snowdf.limit(2).to_pandas()

In [ ]:
oneshort_prompted_snowdf.write.mode("overwrite").save_as_table("AudioConversation_With_Oneshort_PROMPT")

In [ ]:
oneshort_info_extracted_snowdf = oneshort_prompted_snowdf.with_column("EXTRACTED_INFO", 
                                                                      F.call_builtin("SNOWFLAKE.CORTEX.AI_COMPLETE", 
                                                                                     F.lit('claude-3-5-sonnet'), 
                                                                                     F.col("ONESHORT_PROMPT")).cast(StringType()))

In [ ]:
oneshort_info_extracted_snowdf.write.mode('overwrite').save_as_table("TRANSCRIPT_INFO_EXTRACTED_QA")


In [ ]:
session.sql(f"""
insert into AUDIO_CLAIMS_EXTRACTED_INFO(
                                        datetime,
                                        audio_file_name,       
                                        audio_full_file_path,
                                        raw_conversation,
                                        prompted_conversation,
                                        duration,
                                        call_details,
                                        call_summary,
                                        call_sentiment
                                    )
with audio_details as
(
    select datetime, 
            audiofile, 
            audiofile_relative_path, 
            conversation,
            extracted_info,
            duration,
            parse_json(SNOWFLAKE.CORTEX.AI_COMPLETE('claude-3-5-sonnet',concat('<s>[INST] Analyze the audio transcripts found between the tags <question></question> tags and generate only the requested valid json output. Do not include any other language before or after the requested output.Include any policy number or claim number with values like policy number POL_NotAvailable or claim number CL_NotAvailable. Do not include the prompt. Output should only be json and if  ClaimNumber is not found then assign the value as NotFound in double quotes and if there is no PolicyNumber found then assign the value as NotFound in double quotes. 
            Provide a JSON response using the following format, no preamble. Collision with animal cannot be a intent. 
            STRICT REQUIREMENTS:
            - PolicyNumber: Either "NotFound" OR "POL" + digits (e.g., "POL123456")
              ✓ Valid: "POL123456", "POL789012", "NotFound"
              ✗ Invalid: "POL 123-456", "POLL123456", "PAUL 348743"

            - ClaimNumber: Either "NotFound" OR "CLM"+digits (e.g., "CLM123456")
              ✓ Valid: "CLM123456", "CLM789012", "NotFound"
              ✗ Invalid: "CLM 1-23456", "CLM 789012". If you found such claim number , you need to convert it to a valid one as mentioned in above point.
            - ClaimNumber: Either "NotFound" OR "CLM"+digits (e.g., "CLM123456")
            
            CONVERSION RULES:
            - "PAUL 348743" → "POL348743"
            - "POL 123-456" → "POL123456" 
            - "POLL 123456" → "POL123456"
            - "POLL789012" -> "POL789012"
            - "Paul 345678" -> "POL345678"
            - If no valid number found → "NotFound"

            - "CLM 456789" -> "CLM456789"
            - "CLM 789-012" -> "CLM789012"
            
                JSON Schema:
                {{
                  "Representative": "string",
                  "Customer": "string", 
                  "ClaimNumber": "NotFound|CLM[digits]",
                  "PolicyNumber": "NotFound|POL[digits]",
                  "CallIntent": "string (max 2 words)",
                  "CallToAction": "string",
                  "Issue": "string",
                  "Resolution": "string", 
                  "NextSteps": "string",
                  "ModeofUpdate": "string",
                  "PurposeOfCall": "string",
                  "ResponseMode": "string",
                  "FirstCallResolution": 0.8,
                  "CallQuality": "Good|Average|Poor",
                  "Net Promoter Score": 85
                }}
            
            Intent of the call should be in 2 words. Provide if it was a first call resolution as a score values ranging between 0.01 and 0.99.
            Call Quality involves monitoring and evaluating the quality of interactions based on communication skills, adherence to protocols, and overall customer handling and values will be Good,Average,Poor. 
            Net Promoter Score value between 10 -100 and also consider the resolution response time and skill of the Representative in handling the issue.
            Claim number can be like "CLM 8734874" or CLM3473837
            
            Below pattern of conversations is not a first call resolution :
            Owen: Hello, this is Owen from AutoAssure Insurance. How can I assist you today?
            Jessica: Hi, Owen. My name is Jessica Turner, and I am calling to check the status of my insurance claim for a recent car accident.
            Owen: I am here to help, Jessica. Can you please provide me with your policy number so I can access your information?
            Jessica: It is POL901234 or Poll 901234 or Paul 901234.
            Owen: Thank you, Jessica. Let me pull up the details of your claim. While I am doing that, is there something specific you are unclear about regarding the status?
            Jessica: Yes, Owen. I had an accident two weeks ago, and I have not received any updates on the claim.
            Owen: I understand your concern, Jessica. I will investigate the status of your claim and provide you with an update. You will receive an email shortly.
            Jessica: Yes, Owen. How soon can I expect to hear back about the status of my claim?
            Owen: I appreciate your patience, Jessica. Our claims department is actively working on your case, and you can expect an update within the next 48 hours.
            Jessica: Thank you, Owen. I will be waiting for the update. You haveve been very helpful.
            Owen: It is my pleasure, Jessica. If you have any more questions or concerns, feel free to contact us.
            Jessica: Goodbye.
            
            Only provide JSON in your response. Dont provide any explanations.Do not prefix "Here is the JSON response for the conversation" for the json data generated. Value for the Claim Number should be NotFound in double quotes if Claim Number or Policy Number is not found in the text between the <question></question> for all rows.','<question>'
            ,EXTRACTED_INFO,'</question> [/INST] Answer: Here is the JSON response for the conversation')
            )
            ) as call_details 
            
            ,SNOWFLAKE.CORTEX.SUMMARIZE(concat('Provide the summary for the conversation in <con> </con> tag.Generate only the requested output. 
            Do not include any other language before or after the requested output.Also include any policy number or claim number if found the conversation. Do not include the prompt. Dont provide any explanations.
            <con>',CONVERSATION,'</con>')) as call_summary
            
            , SNOWFLAKE.CORTEX.AI_SENTIMENT(CONVERSATION) as call_sentiment
            
FROM TRANSCRIPT_INFO_EXTRACTED_QA
)
SELECT * FROM audio_details

""").collect()

In [ ]:
session.sql(f'''

CREATE OR REPLACE TABLE STREAMLITAPPTABLE
As
select 
 datetime 
,audio_file_name
,audio_full_file_path
,duration
,call_details:CallToAction::string as CallToAction
,call_details:ClaimNumber::string as ClaimNumber
,call_details:Customer::string as Customer
,call_details:CallIntent::string as Intent
,call_details:Issue::string as Issue
,call_details:PolicyNumber::string as PolicyNumber
,call_details:PurposeOfCall::string as PurposeOfCall
,call_details:Representative::string as Representative
,call_details:Resolution::string as Resolution
,call_details:ResponseMode::string as ResponseMode
,call_details:ModeofUpdate::string as ModeofUpdate
,call_details:NextSteps::string as NextSteps
,IFF(call_sentiment:categories[0]['sentiment']='Positive','Positive','Negative') as CallSentiment
,IFF(call_details:FirstCallResolution>0.5,'Yes','No') as FirstCallResolution
,call_summary
,prompted_conversation  as call_diarization
from
Audio_Claims_Extracted_info
            
            ''').collect()

In [ ]:
CREATE OR REPLACE PROCEDURE PROCESS_UPLOADED_AUDIO(file_path STRING DEFAULT NULL)
    RETURNS STRING
    LANGUAGE PYTHON
    RUNTIME_VERSION = '3.10'
    PACKAGES = ('snowflake-snowpark-python')
    HANDLER = 'process_audio'
    AS
    $$
import snowflake.snowpark.functions as F
from snowflake.snowpark import Session
from snowflake.snowpark.types import StringType,IntegerType

def process_audio(session: Session, file_path: str = None):
    try:
        
        session.sql('alter stage AUDIO_FILES refresh').collect()
        
        # Step 1: Insert into ALL_CLAIMS_RAW
        insert_sql = """
        INSERT INTO ALL_CLAIMS_RAW
        (
            DATETIME,
            AUDIOFILE,
            AUDIOFILE_RELATIVE_PATH,
            CONVERSATION,
            DURATION,
            PRESIGNED_URL_PATH
            
        )
        SELECT 
            CAST(CASE WHEN split(RELATIVE_PATH,'/')[1]::string IS NULL THEN GETDATE() 
                      ELSE split(RELATIVE_PATH,'/')[0]::string END AS DATE) as Datetime,
            CASE WHEN split(RELATIVE_PATH,'/')[1]::string is null then split(RELATIVE_PATH,'/')[0]::string 
                  ELSE split(RELATIVE_PATH,'/')[1]::string END as AudioFile,
            RELATIVE_PATH AS AUDIOFILE_RELATIVE_PATH,
            AI_TRANSCRIBE(TO_FILE('@AUDIO_FILES', RELATIVE_PATH))['text']::varchar AS EXTRACTED_TEXT,
            AI_TRANSCRIBE(TO_FILE('@AUDIO_FILES', RELATIVE_PATH))['audio_duration']::varchar AS DURATION_IN_SECONDS,
            GET_PRESIGNED_URL('@AUDIO_FILES', RELATIVE_PATH) AS PRESIGNED_URL
        FROM DIRECTORY('@AUDIO_FILES') """ + (
            "WHERE relative_path = '" + file_path + "'" if file_path and file_path.strip() else ""
        ) + """
        """
        
        # Execute the insert
        session.sql(insert_sql).collect()
        
        # Step 2: Process with Snowpark DataFrames
        generated_transcripts_df = session.table("ALL_CLAIMS_RAW")
        if file_path and file_path.strip():
            generated_transcripts_df = generated_transcripts_df.filter(F.col("audiofile_relative_path") == file_path)
        
        # Apply oneshort prompt UDF
        oneshort_prompted_snowdf = generated_transcripts_df.with_column(
            "Oneshort_PROMPT", 
            F.call_udf("get_qa_prompt_oneshort", F.col("CONVERSATION"))
        )
        
        # Save to intermediate table
        oneshort_prompted_snowdf.write.mode("append").save_as_table("AudioConversation_With_Oneshort_PROMPT")
        
        # Extract information using another UDF (assuming this is the missing UDF call)
        oneshort_info_extracted_snowdf = oneshort_prompted_snowdf.with_column("EXTRACTED_INFO", 
                                                                      F.call_builtin("SNOWFLAKE.CORTEX.AI_COMPLETE", 
                                                                                     F.lit('claude-3-5-sonnet'), 
                                                                                     F.col("ONESHORT_PROMPT")).cast(StringType()))
        
        # Save final results
        oneshort_info_extracted_snowdf.write.mode('append').save_as_table("TRANSCRIPT_INFO_EXTRACTED_QA")

        final_sql = """
            insert into AUDIO_CLAIMS_EXTRACTED_INFO(
                                                    datetime,
                                                    audio_file_name,       
                                                    audio_full_file_path,
                                                    raw_conversation,
                                                    prompted_conversation,
                                                    duration,
                                                    call_details,
                                                    call_summary,
                                                    call_sentiment
                                                        )
                    SELECT  datetime, 
                            audiofile, 
                            audiofile_relative_path, 
                            conversation,
                            extracted_info,
                            duration,
                            -- Complete AI prompt for call details extraction
                            parse_json(SNOWFLAKE.CORTEX.AI_COMPLETE('claude-3-5-sonnet',
                                concat('Extract call information as JSON from this conversation. Return ONLY valid JSON with all fields. Required format: {"CallIntent": "2 words max", "CallQuality": "Good/Average/Poor", "CallToAction": "brief action description", "ClaimNumber": "CLM123456 or NotFound", "Customer": "customer name", "FirstCallResolution": 0.8, "Issue": "brief issue description", "ModeofUpdate": "Email/Phone/Portal", "Net Promoter Score": 85, "NextSteps": "brief next steps", "PolicyNumber": "POL123456 or NotFound", "PurposeOfCall": "brief purpose", "Representative": "rep name", "Resolution": "brief resolution", "ResponseMode": "Follow-up method"}. Extract from conversation: ', EXTRACTED_INFO)
                            )) as call_details,
                            
                            -- Simplified summary prompt  
                            SNOWFLAKE.CORTEX.SUMMARIZE(concat('Summarize this call center conversation in 2-3 sentences: ', CONVERSATION)) as call_summary,
                            
                            -- Sentiment analysis 
                            SNOWFLAKE.CORTEX.AI_SENTIMENT(CONVERSATION) as call_sentiment
                            
                    FROM TRANSCRIPT_INFO_EXTRACTED_QA """ + (
            "WHERE audiofile_relative_path = '" + file_path + "'" if file_path and file_path.strip() else ""
        ) + """
        """
        
        session.sql(final_sql).collect()
        
        # Step 3: Insert into Streamlit App Table
        streamlit_sql = """
            INSERT INTO STREAMLITAPPTABLE (
                datetime,
                audio_file_name,
                audio_full_file_path,
                duration,
                CallToAction,
                Intent,
                ClaimNumber,
                Customer,
                FirstCallResolution,
                Issue,
                ModeofUpdate,
                NextSteps,
                PolicyNumber,
                PurposeOfCall,
                Representative,
                Resolution,
                ResponseMode,
                CallSentiment,
                call_summary,
                call_diarization
            )
            SELECT 
                datetime,
                audio_file_name,
                audio_full_file_path,
                duration,
                call_details:CallToAction::string as CallToAction,
                call_details:CallIntent::string as Intent,
                call_details:ClaimNumber::string as ClaimNumber,
                call_details:Customer::string as Customer,
                IFF(call_details:FirstCallResolution::float > 0.5, 'Yes', 'No') as FirstCallResolution,
                call_details:Issue::string as Issue,
                call_details:ModeofUpdate::string as ModeofUpdate,
                call_details:NextSteps::string as NextSteps,
                call_details:PolicyNumber::string as PolicyNumber,
                call_details:PurposeOfCall::string as PurposeOfCall,
                call_details:Representative::string as Representative,
                call_details:Resolution::string as Resolution,
                call_details:ResponseMode::string as ResponseMode,
                IFF(call_sentiment:categories[0]['sentiment']='positive','Positive','Negative') as CallSentiment,
                call_summary,
                prompted_conversation as call_diarization
            FROM AUDIO_CLAIMS_EXTRACTED_INFO """ + (
            "WHERE audio_full_file_path = '" + file_path + "'" if file_path and file_path.strip() else ""
        ) + """
        """
        
        session.sql(streamlit_sql).collect()
        
        if file_path and file_path.strip():
            return "Successfully processed audio file and populated Streamlit table: " + file_path
        else:
            return "Successfully processed all audio files and populated Streamlit table"

        
    except Exception as e:
        error_context = file_path if file_path and file_path.strip() else "all files"
        return "Error processing " + error_context + ": " + str(e)
$$;

In [ ]:
-- call PROCESS_UPLOADED_AUDIO('2025-08-08/ztest_5.mp3');

In [ ]:
select * from all_claims_raw order by datetime desc;

-- ls @audio_files;

In [ ]:
select * from TRANSCRIPT_INFO_EXTRACTED_QA order by datetime desc limit 40;

In [ ]:
select prompted_conversation,* from AUDIO_CLAIMS_EXTRACTED_INFO order by datetime desc, audio_file_name;

-- delete from AUDIO_CLAIMS_EXTRACTED_INFO  where datetime='2025-08-07'

In [ ]:
-- drop table Call_Center_Interactions;

select * from STREAMLITAPPTABLE order by datetime desc;

In [ ]:
CREATE OR REPLACE TABLE PUBLIC.AUDIO_CLAIMS_SEARCH_TABLE AS
SELECT 
    AUDIO_FILE_NAME,
    DATETIME,
    DURATION as CALL_DURATION_IN_SECONDS,
    RAW_CONVERSATION,
    PROMPTED_CONVERSATION,
    CALL_DETAILS,
    CALL_SUMMARY,
    CALL_SENTIMENT,
    REPONSE_GIVEN,
    -- Create a comprehensive searchable text column combining all relevant information
    CONCAT(
        'Call Date: ', COALESCE(DATETIME::STRING, 'Unknown'), '\n',
        'Audio File: ', COALESCE(AUDIO_FILE_NAME, 'Unknown'), '\n',
        'Call Duration in Seconds: ', COALESCE(DURATION::STRING, '0'), ' seconds\n',
        'Call Summary: ', COALESCE(CALL_SUMMARY, 'No summary available'), '\n',
        'Conversation: ', COALESCE(PROMPTED_CONVERSATION, RAW_CONVERSATION, 'No conversation available'), '\n',
        'Call Details: ', COALESCE(CALL_DETAILS::STRING, 'No details available'), '\n',
        'Sentiment: ', COALESCE(CALL_SENTIMENT::STRING, 'No sentiment data')
    ) AS SEARCHABLE_CONTENT
FROM AUDIO_CLAIMS_EXTRACTED_INFO
WHERE PROMPTED_CONVERSATION IS NOT NULL 
   OR RAW_CONVERSATION IS NOT NULL;

-- Create search service on the table
CREATE OR REPLACE CORTEX SEARCH SERVICE PUBLIC.CALL_CENTRE_SEARCH_SERVICE
ON SEARCHABLE_CONTENT
ATTRIBUTES AUDIO_FILE_NAME, DATETIME, CALL_DURATION_IN_SECONDS, CALL_SUMMARY, CALL_SENTIMENT_STR, REPONSE_GIVEN
WAREHOUSE = COMPUTE_WH
TARGET_LAG = '1 hour'
AS (
    SELECT 
        AUDIO_FILE_NAME,
        DATETIME,
        CALL_DURATION_IN_SECONDS,
        CALL_SUMMARY,
        CALL_SENTIMENT::STRING AS CALL_SENTIMENT_STR,  -- Convert VARIANT to STRING
        REPONSE_GIVEN,
        SEARCHABLE_CONTENT
    FROM CALL_CENTRE_AI_TRANSCIBE.PUBLIC.AUDIO_CLAIMS_SEARCH_TABLE
);

In [ ]:
drop CORTEX SEARCH SERVICE PUBLIC.CALL_CENTRE_SEARCH_SERVICE_TABLE;

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'claude-4-sonnet',
    (SELECT 
        parse_json(SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
            'CALL_CENTRE_AI_TRANSCIBE.PUBLIC.CALL_CENTRE_SEARCH_SERVICE',
            '{
                "query": "what questions were asked by the rep Quinn for the call happened on March 13 2025 on the policy number 123456? Is the customer satisfied with the answers given? If not, what better questions could the rep have asked?",
                "columns": ["AUDIO_FILE_NAME","SEARCHABLE_CONTENT"],
                "limit": 1
            }'
        ))['results'][0]['SEARCHABLE_CONTENT']::string
    )
);




In [ ]:
session.sql(''' 
CREATE or replace TABLE PolicyDetails (
    PolicyID                INT IDENTITY,
    PolicyNumber            VARCHAR(50) UNIQUE,
    CustomerID              INT,
    PolicyType              VARCHAR(50),
    CoverageAmount          DECIMAL(10, 2),
    PremiumAmount           DECIMAL(8, 2),
    StartDate               DATE,
    EndDate                 DATE,
    RenewalDate             DATE,
    PolicyStatus            VARCHAR(20),
    PaymentStatus           VARCHAR(20),
    Beneficiaries           VARCHAR(255),
    CoverageDetails         TEXT,
    RidersEndorsements      TEXT,
    InsuredItemsPersons     TEXT,
    PolicyholderDetails     TEXT,
    AgentBrokerDetails      TEXT,
    Deductibles             DECIMAL(8, 2),
    Exclusions              TEXT,
    ClaimsHistory           TEXT,
    UnderwritingDetails     TEXT,
    PolicyTermsConditions   TEXT
); ''').collect()

In [ ]:
session.sql(''' 
INSERT INTO PolicyDetails (     
PolicyNumber         
,CustomerID           
,PolicyType           
,CoverageAmount       
,PremiumAmount        
,StartDate            
,EndDate              
,RenewalDate          
,PolicyStatus         
,PaymentStatus        
,Beneficiaries        
,CoverageDetails      
,RidersEndorsements   
,InsuredItemsPersons  
,PolicyholderDetails  
,AgentBrokerDetails   
,Deductibles          
,Exclusions           
,ClaimsHistory        
,UnderwritingDetails  
,PolicyTermsConditions) VALUES
( 'POL001', 1001, 'Auto', 500000, 1000, '2022-01-01', '2023-01-01', '2022-12-31', 'Active', 'Paid', 'John Doe', 'Basic coverage for vehicle insurance.', 'None', 'John Doe, Jane Doe', 'John Doe, 123 Insurance Inc.', 'John Doe, Jane Doe', 100, 'None', 'No claims made yet.', 'Standard underwriting process completed.', 'Standard terms and conditions apply.'),
( 'POL002', 1002, 'Auto', 100000, 500, '2022-02-01', '2023-02-01', '2023-01-31', 'Active', 'Due', 'Jane Doe', 'Basic coverage for health insurance.', 'Critical Illness Rider', 'Jane Doe, Family Members', 'Jane Doe, 456 Insurance Services', 'Jane Doe, Family Members', 50, 'Pre-existing conditions', 'No claims made yet.', 'Medical examination completed.', 'Standard terms and conditions apply.'),
( 'POL010', 1010, 'Auto', 25000, 300, '2022-10-01', '2023-10-01', '2023-09-30', 'Active', 'Paid', 'Alice Smith', 'Basic coverage for auto insurance.', 'Roadside Assistance Rider', 'Alice Smith, Vehicle Details', 'Alice Smith, XYZ Insurance Agency', 'Alice Smith, Vehicle Details', 75, 'Accidents caused by negligence', 'No claims made yet.', 'Vehicle inspection completed.', 'Standard terms and conditions apply.'),
( 'POL934514', 1007, 'Auto', 500000, 1000, '2023-08-01', '2024-08-01', '2024-07-25', 'Active', 'Paid', 'Mike S', 'Full coverage for vehicle insurance.', 'None', 'Mike S', 'Mike S, 456 Insurance Inc.', 'Mike S, 
Jane S', 100, 'None', '2 claims made so far.', 'Standard underwriting process completed.', 'Standard terms and conditions apply.')

''').collect()

In [ ]:
session.sql('''
CREATE or replace TABLE AutoClaims (
    ClaimID varchar(50) ,
    PolicyID INT,
    ClaimDate DATE,
    ClaimAmount DECIMAL(10, 2),
    ClaimDescription TEXT,
    ClaimStatus VARCHAR(50)
)
''').collect()

In [ ]:
session.sql('''
INSERT INTO AutoClaims (ClaimID,PolicyID, ClaimDate, ClaimAmount, ClaimDescription, ClaimStatus)
VALUES
    ('CLM234567',4, '2023-09-15', 5000, 'Vehicle collision at an intersection.', 'Pending'),
    ('CLM876543',5, '2023-04-20', 1200, 'Minor fender bender in a parking lot.', 'Approved'),
    ('CLM567890',6, '2023-10-10', 8000, 'Total loss due to a major accident on the highway.', 'Denied'),
    ('CLM345678901',7, '2023-09-15', 5000, 'Vehicle Damage', 'Pending')
''').collect()